In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-chi-merged/yelp_CHI.csv
/kaggle/input/reviews-by-phi2/reviews_by_phi_2_v_2.txt
/kaggle/input/reviews-by-phi2/reviews_by_phi_2_v_1.txt
/kaggle/input/reviews-by-phi2/reviews_by_phi_2_v_3.txt


In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/word

In [3]:
! pip install contractions
import re
import contractions

stop_words = set(stopwords.words('english'))
def preprocess_review(text):
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub(r"[^a-zA-Z'\s]", '', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

text = "1. This is an example for wifes test 145 13 we're don't"
preprocess_review(text)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.2 MB/s eta 0:00:00


'example wifes test'

In [4]:
import os

path = '/kaggle/input/reviews-by-phi2'
fnames = os.listdir(path)
fpaths = [os.path.join(path, fname) for fname in fnames]
fpaths

['/kaggle/input/reviews-by-phi2/reviews_by_phi_2_v_2.txt',
 '/kaggle/input/reviews-by-phi2/reviews_by_phi_2_v_1.txt',
 '/kaggle/input/reviews-by-phi2/reviews_by_phi_2_v_3.txt']

In [5]:
lst_reviews = []

for fp in fpaths:
    with open(fp, 'r', encoding='utf-8') as file:
        reviews = file.readlines()
        lst_reviews = lst_reviews + reviews
        print(len(lst_reviews))

12811
26165
34569


In [6]:
df = pd.read_csv('/kaggle/input/yelp-chi-merged/yelp_CHI.csv')[['label', 'review']]
df

,label,review
0,1,"Unlike Next, which we'd eaten at the previous ..."
1,1,Probably one of the best meals I've had ever. ...
2,1,Service was impeccable. Experience and present...
3,1,"The problem with places like this, given the e..."
4,1,I have no idea how to write my review - dining...
...,...,...
61533,1,I've passed this place a bunch of times on my ...
61534,1,Rubino's is not really setup as a retail opera...
61535,1,This place is amazing. The food is great and y...
61536,1,OK went back the next day as I was driving by ...


In [7]:
df['label'].value_counts()

label
 1    53397
-1     8141
Name: count, dtype: int64

In [8]:
df_generated_reviews = pd.DataFrame(lst_reviews)
df_generated_reviews.columns = ['review']
df_generated_reviews['label'] = -1

In [9]:
df = pd.concat([df, df_generated_reviews], axis=0)
df

,label,review
0,1,"Unlike Next, which we'd eaten at the previous ..."
1,1,Probably one of the best meals I've had ever. ...
2,1,Service was impeccable. Experience and present...
3,1,"The problem with places like this, given the e..."
4,1,I have no idea how to write my review - dining...
...,...,...
34564,-1,"1. ""I had lunch at this new restaurant in my ..."
34565,-1,"2. ""This restaurant is perfect for a quick lun..."
34566,-1,"3. ""I ordered delivery from this restaurant an..."
34567,-1,"4. ""I had lunch at this new restaurant in my o..."


In [10]:
df['label'].value_counts()

label
 1    53397
-1    42710
Name: count, dtype: int64

In [11]:
df.loc[:, 'review'] = df['review'].apply(preprocess_review)

In [12]:
df['review'].duplicated().sum()

2178

In [13]:
df = df.drop_duplicates()
df.duplicated().sum()

0

In [14]:
df['label'].value_counts()

label
 1    53367
-1    40587
Name: count, dtype: int64

In [15]:
df['label'] = df['label'].replace(-1, 0)
df

<ipython-input-15-52d9f656d55d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].replace(-1, 0)


,label,review
0,1,unlike next would eaten previous night dish co...
1,1,probably one best meals ever performance food ...
2,1,service impeccable experience presentation coo...
3,1,problem places like given exhorbitant cost med...
4,1,idea write review dining alinea brings whole d...
...,...,...
34564,0,lunch new restaurant old neighborhood great pr...
34565,0,restaurant perfect quick lunch dinner food rea...
34566,0,ordered delivery restaurant great experience f...
34567,0,lunch new restaurant old neighborhood great ex...


In [16]:
df = df.sample(frac=1).reset_index(drop=True)

In [17]:
df

,label,review
0,1,dear calo thank always consistent really nice ...
1,1,get wrong everything good tasty feel like put ...
2,1,lovely place great atmosphere wonderful staff ...
3,0,sushi night highly recommend new place downtow...
4,0,highly recommend italian restaurant pasta dish...
...,...,...
93949,1,place awesome first love coffee want great cup...
93950,1,boom boom boom let us go back room eat night m...
93951,1,repeating everyone else saidit amazing went th...
93952,1,review good pork sandwich lots flavor deliciou...


In [18]:
df.to_csv('yelp_chi_with_generated_reviews.csv', index=False)